In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
import pandas as pd

In [2]:
#adjust file paths
path_train='/media/salah/01D694C967465C10/CV_Internship/datasets/train_folder'
path_test= '/media/salah/01D694C967465C10/CV_Internship/datasets/test_folder'
train_smile = os.path.join(path_train , '1')
train_not_smile=os.path.join(path_train , '0')
test_smile = os.path.join(path_test , '1')
test_not_smile=os.path.join(path_test , '0')

In [8]:
#smile
train_smile_files= os.listdir(train_smile)
images= []
for img in train_smile_files:
    img_path = os.path.join(train_smile,img)
    imgg = np.array(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE))
    images.append(imgg.ravel())
images=np.array(images)
smile_df = pd.DataFrame(images)
smile_df['target']=1



#no smile
train_not_smile_files= os.listdir(train_not_smile)
images_not= []
for img in train_not_smile_files:
    img_path = os.path.join(train_not_smile,img)
    imgg = np.array(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE))
    images_not.append(imgg.ravel())
images_not=np.array(images_not)
not_smile_df = pd.DataFrame(images_not)
not_smile_df['target']=0

train_df = smile_df.append(not_smile_df)
train_df=train_df.sample(frac = 1) 
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,target
246,14,13,13,15,17,18,19,19,20,21,...,14,11,13,13,10,8,10,11,11,0
203,70,63,59,43,30,27,18,10,10,11,...,10,9,9,9,9,10,10,10,10,1
660,135,131,133,139,147,147,152,164,167,167,...,3,1,1,1,1,1,1,1,1,1
584,181,181,181,182,183,182,180,178,173,176,...,215,216,219,214,218,220,216,215,205,0
1430,92,89,86,85,81,80,84,82,84,86,...,25,25,25,25,27,32,36,37,36,1


In [9]:
train_df.shape

(2800, 4097)

In [10]:
#test files
test_smile_files= os.listdir(test_smile)
images_t= []
for img in test_smile_files:
    img_path = os.path.join(test_smile,img)
    imgg = np.array(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE))
    images_t.append(imgg.ravel())
images_t=np.array(images_t)
smile_df = pd.DataFrame(images_t)
smile_df['target']=1




test_not_smile_files= os.listdir(test_not_smile)
images_not_t= []
for img in test_not_smile_files:
    img_path = os.path.join(test_not_smile,img)
    imgg = np.array(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE))
    images_not_t.append(imgg.ravel())
images_not_t=np.array(images_not_t)
not_smile_df = pd.DataFrame(images_not_t)
not_smile_df['target']=0

test_df = smile_df.append(not_smile_df)
test_df=test_df.dropna()
test_df=test_df.sample(frac = 1) 
test_df.head()


,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,target
554,1,3,2,0,2,5,16,18,26,44,...,17,15,16,15,15,15,17,14,15,0
139,50,50,50,49,50,51,50,48,51,49,...,73,76,94,85,92,92,81,80,79,1
280,37,40,36,38,38,40,39,39,36,42,...,174,179,183,183,182,181,177,170,165,1
129,43,48,43,38,76,88,95,105,126,135,...,246,252,249,249,242,243,225,200,129,1
521,87,90,94,98,102,103,105,109,143,141,...,214,213,213,211,210,210,211,212,212,1


In [11]:
test_df.shape

(1200, 4097)

In [12]:
x_train = train_df.iloc[:,0:4096]
y_train = train_df['target']
x_test = test_df.iloc[:,0:4096]
y_test = test_df['target']

In [18]:
#model
from sklearn.svm import SVC
svc = SVC(kernel='rbf')


In [19]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print ("Accuracy on training set:")
    print (clf.score(X_train, y_train))
    print ("Accuracy on testing set:")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("Classification Report:")
    print (metrics.classification_report(y_test, y_pred))
    print ("Confusion Matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))

In [20]:
train_and_evaluate(svc, x_train, x_test, y_train, y_test)

Accuracy on training set:
0.885
Accuracy on testing set:
0.7791666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.72      0.77       600
           1       0.75      0.83      0.79       600

    accuracy                           0.78      1200
   macro avg       0.78      0.78      0.78      1200
weighted avg       0.78      0.78      0.78      1200

Confusion Matrix:
[[435 165]
 [100 500]]


In [26]:
from joblib import dump, load

dump(svc,'smile_d.sav')

['smile_d.sav']

In [103]:
svc= load('smile_d.sav')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
vid = cv2.VideoCapture(0)
l= ['not smile','smile']
while True:
    ret, img = vid.read()
    cv2.waitKey(1)
    if not ret:
        break
    faces =  face_cascade.detectMultiScale(img, 1.1, 4)[0]
    (x, y, w, h) = faces
    roi = img[y:y+h, x:x+w]
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi,(64,64))
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    prediction=svc.predict(roi.ravel().reshape(1,-1))
    cv2.putText(img,'{}'.format(l[int(prediction)]),(x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, 155, 10)
    cv2.imshow('img', img)
    
vid.release()    
    
    

IndexError: tuple index out of range